# The shallow water equations

In this notebook, we'll move to a more interesting problem: the shallow water equations.
The shallow water equations are one of the simplest mathematical models in geophysical fluid dynamics.
Nonetheless, they exhibit a very large share of the complexities of more realistic physics models like the Boussinesq equations.
They are a much more powerful test for the quality of spatial and temporal discretization schemes than the scalar advection equation.

The most basic form of the shallow water system describes the evolution of the thickness $h$ and velocity $u$ of an incompressible fluid flowing over a bed with elevation $b$.
They are:

$$\begin{align}
\partial_th + \nabla\cdot hu & = 0 \\
\partial_thu + \nabla\cdot\left\{hu\otimes u + \frac{1}{2}gh^2I\right\} & = -gh\nabla b
\end{align}$$

where $g$ is the acceleration due to gravity and $I$ is the 2D identity matrix.
The divergence in the second equation should be thought of as the divergence of a tensor field.
The symbol "$\otimes$" denotes the *outer product*.
You can think of the outer product of two vectors as the matrix $u\cdot u^\top$, where $\top$ denotes the transpose.
Alternatively, if you like indices, the components of the outer product are $(u \otimes u)_{ij} = u_iu_j$.

This form of the shallow water equations has an aggravating difficulty in that the time derivative is really of the *momentum*.
In this form of the problem, the momentum is a non-trivial derived function of the thickness and velocity.
It's possible to design around this when coding up numerical solvers but it is extremely annoying and as a consequence we'll instead work with the thickness and the momentum

$$q = hu$$

to arrive at the system

$$\begin{align}
\partial_th + \nabla\cdot q & = 0 \\
\partial_tq + \nabla\cdot\left\{h^{-1}q\otimes q + \frac{1}{2}gh^2I\right\} & = -gh\nabla b.
\end{align}$$

In this equivalent form of the problem, the time derivative is on just the momentum and not some function of the momentum and other variables.
We can then easily implement common timestepping schemes.

### Problem setup

First, we'll use a periodic rectangle as our spatial domain of 20m to a side -- a little less than the width of an Olympic swimming pool.
The periodicity is a little artificial and in later demos we'll show how to add realistic boundary conditions.

In [ ]:
import firedrake
nx, ny = 32, 32
Lx, Ly = 20., 20.
mesh = firedrake.PeriodicRectangleMesh(
    nx, ny, Lx, Ly, diagonal='crossed'
)

As an initial condition, we'll take the water to be 1m deep, and to perturb this a bit we'll add a 10cm parabolic burp near one corner.

In [ ]:
from firedrake import inner, max_value, Constant
x = firedrake.SpatialCoordinate(mesh)
lx = 5.
y = Constant((lx, lx))
r = Constant(2.5)

H = Constant(1.)
δh = Constant(0.1)
h_expr = H + δh * max_value(0, 1 - inner(x - y, x - y) / r**2)

To make things yet more interesting, we'll add some variable bottom topography consisting of another parabolic burp.

In [ ]:
y = Constant((3 * lx, 3 * lx))
δb = Constant(1/4)
b = δb * max_value(0, 1 - inner(x - y, x - y) / r**2)

This is all the input data we need.

### DG(1) discretization

In the last demo, we saw that using a higher-resolution scheme than DG(0) gave a huge improvement in the accuracy of the solution.
Here we'll do the same thing and start with a DG(1) discretization for both thickness and momentum.
For the momentum, we'll create a vector rather than a scalar function space.

In [ ]:
degree = 1
Q = firedrake.FunctionSpace(mesh, 'DG', degree)
V = firedrake.VectorFunctionSpace(mesh, 'DG', degree)

As we saw in the last demo, the solvers are expecting to work with one single monolithic state vector.
What we want is the cartesian product $Z = Q \times V$ of the thickness space $Q$ and the momentum space $V$, and Firedrake has some built-in notation for doing just that.

In [ ]:
Z = Q * V
z_0 = firedrake.Function(Z)

In order to initialize the state vector $z_0$, we can split out the thickness and momentum components and project the expression we wrote above.

In [ ]:
h_0, q_0 = z_0.split()
h_0.project(h_expr - b);

Now we'll do just like we did before: create a function that will return the right-hand side of the shallow water equations and then create a numerical solver for those equations.

In [ ]:
from plumes import models
from plumes.coefficients import gravity
g = firedrake.Constant(gravity)
equation = models.shallow_water.make_equation(g, b)

The wave speed for the shallow water equations is $\sqrt{g\cdot h}$, which gives us a speed to guess at a CFL-stable timestep with.

In [ ]:
import numpy as np
C = np.sqrt(gravity * (float(H) + float(δh)))
δx = mesh.cell_sizes.dat.data_ro[:].min()
timestep = (δx / 8) / C / (2 * degree + 1)
print(timestep)

We can also use this to guess how long it will take for a wave to propagate all the way across the domain.

In [ ]:
final_time = 4 * Lx / C
num_steps = int(final_time / timestep)
print(f'final time: {final_time:5.3f}')
print(f'num steps:  {num_steps}')
dt = final_time / num_steps

We'll want to make movies again, so we need to calculate how the output frequency.

In [ ]:
output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)
print(f'output frequency: {output_freq}')

Now we can make our solver and proceed like before.

In [ ]:
from plumes import numerics
integrator = numerics.ExplicitEuler(equation, z_0)

In the timestepping loop, we'll add a bit of diagnostic information to the progress bar to make sure the simulation hasn't exploded.

In [ ]:
import tqdm

hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

With the saved time series, we'll make a movie of the surface elevations throughout the simulation.
We're going to make several movies, so this is worth factoring out into a separate function.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

def setup_movie(hs, η, vmin, vmax, output_freq):
    fig, axes = plt.subplots()
    axes.set_aspect('equal')
    axes.get_xaxis().set_visible(False)
    axes.get_yaxis().set_visible(False)
    axes.set_xlim((0, Lx))
    axes.set_ylim((0, Ly))
    colors = firedrake.tripcolor(
        η, num_sample_points=1, vmin=vmin, vmax=vmax, axes=axes
    )

    def animate(h):
        η.project(h + b)
        colors.set_array(η.dat.data_ro[:])

    interval = 1e3 * output_freq * dt
    animation = FuncAnimation(fig, animate, frames=hs, interval=interval)
    return animation

In [ ]:
%%capture
Q0 = firedrake.FunctionSpace(mesh, 'DG', 0)
η = firedrake.project(hs[0] + b, Q0)
animation = setup_movie(hs, η, 0.85, 1.05, output_freq)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

Notice how, at the end of the movie, there are some high-frequency oscillations?
Those should make you suspicious that something isn't entirely right with the numerics.
We can quantify this by looking at the degree to which the numerical solution conserves energy.
The energy functional for the shallow water equations is

$$E = \frac{1}{2}\int_\Omega\left\{h^{-1}|q|^2 + g\cdot(h + b)\right\}dx.$$

Let's plot the value of the energy functional over the whole time series:

In [ ]:
from firedrake import assemble, dx
energies = np.array([
    0.5 * assemble((inner(q, q) / h + g * (h + b)) * dx)
    for h, q in zip(hs, qs)
])

In [ ]:
fig, axes = plt.subplots()
times = dt * output_freq * np.array(list(range(num_steps // output_freq)))
axes.plot(times, energies)
axes.set_xlabel('time (s)')
axes.set_ylabel('energy');

The simulated state is clearly gaining energy over time and will, eventually, become unstable.
This is an expected and well-known property of the explicit Euler scheme -- it tends to "inject" energy into otherwise energy-conserving systems.
We could tamp down this effect over the integration period by reducing the timestep.
But this failure of energy conservation can only be eliminated by using better integration schemes.

In order to facilitate comparing all of the different schemes at the end, we'll store the time series in a dictionary.

In [ ]:
results = {
    'explicit Euler, DG(1)': (hs, qs)
}

### SSPRK3 scheme

Rather than use an even shorter timestep, we can partly reduce the energy error by going to a higher-order time discretization.
Here we'll use the 3-stage, 3rd-order *strong stability-preserving Runge-Kutta method* or SSPRK3/3 for short.
This scheme is very common in computational fluid dynamics because it gives 3rd-order accuracy in time very economically and with numerically simple coefficients.
Since the exact system conserves energy, a time discretization that better tracks the true solution should also track the energy better.

In [ ]:
integrator = numerics.SSPRK33(equation, z_0)

hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

We'll save the results for later:

In [ ]:
results['SSPRK3/3, DG(1)'] = (hs, qs)

Now we can calculate the energy drift again:

In [ ]:
energies = np.array([
    0.5 * assemble((inner(q, q) / h + g * (h + b)) * dx)
    for h, q in zip(hs, qs)
])

In [ ]:
fig, axes = plt.subplots()
times = dt * output_freq * np.array(list(range(num_steps // output_freq)))
axes.plot(times, energies)
axes.set_xlabel('time (s)')
axes.set_ylabel('energy');

Using SSPRK3 has reduced the energy drift by a factor of 10.
But we should emphasize that it does *not* eliminate the problem, it merely reduces it.
To get true energy conservation, we would need to use a symplectic scheme like the implicit midpoint rule.
Nonetheless, it's a big improvement and much more economical than using explicit Euler and cutting the timestep.

Now let's make another movie to see if there's any other weird stuff sticking out.

In [ ]:
%%capture
animation = setup_movie(hs, η, 0.85, 1.05, output_freq)

In [ ]:
HTML(animation.to_html5_video())

The oscillations that we saw before are largely gone, but something weird is happening in the vicinity of the bed bump.
Let's see if we can improve on that by changing the spatial discretization.

### BDFM discretization

SSPRK3 helped substantially with the energy drift, but there are still some artifacts around where there are sources of momentum from bottom topography.
The way that this effect usually manifests itself is in a solution that starts with a flat surface, which should be steady, having non-trivial evolution.
We can improve things by making a better choice of finite element space.
Here, we'll use the degree-2 Brezzi-Douglas-Fortin-Marini element, or BDFM(2) for short.
The pair of BDFM(2) for momentum and DG(1) for thickness on triangular elements reproduces many of the favorable properties of staggered finite difference grids.

In [ ]:
degree = 2
V = firedrake.FunctionSpace(mesh, 'BDFM', degree)

Z = Q * V
z_0 = firedrake.Function(Z)

h_0, q_0 = z_0.split()
h_0.project(h_expr - b);

In [ ]:
timestep = (δx / 8) / C / (2 * degree + 1)
num_steps = int(final_time / timestep)
print(f'final time: {final_time:5.3f}')
print(f'num steps:  {num_steps}')
dt = final_time / num_steps

output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)
print(f'output frequency: {output_freq}')

In [ ]:
integrator = numerics.SSPRK33(equation, z_0)

In [ ]:
hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

In [ ]:
results['SSPRK3/3, BDFM(2)'] = (hs, qs)

Now let's look at the energy drift and make a movie again.

In [ ]:
Es = np.array([
    0.5 * assemble((inner(q, q) / h + g * (h + b)) * dx)
    for h, q in zip(hs, qs)
])
Es.min(), Es.max()

In [ ]:
fig, axes = plt.subplots()
axes.plot(Es);

In [ ]:
η = firedrake.project(hs[0] + b, Q0)

In [ ]:
%%capture
animation = setup_movie(hs, η, 0.85, 1.05, output_freq)

In [ ]:
HTML(animation.to_html5_video())

The energy drift is further reduced by 1/3, but more importantly, the visual artifacts in the solution near the topographic bump have been substantially reduced.

The BDFM(2)/DG(1) element pair with SSPRK3 timestepping is appreciably more expensive than DG(1)/DG(1) with explicit Euler on the same grid.
For realistic problems, we are free to choose whichever grid we want and are instead constrained by computational resources.

### Another SSPRK3 scheme

The SSPRK3/3 scheme is much more accurate than explicit Euler, and it would be convenient if we could use a longer timestep to get greater accuracy at less cost.
Unfortunately, the SSPRK3/3 scheme also has the same stability requirements as explicit Euler, so increasing the timestep isn't an option.
We can make the desired tradeoff by using a 3rd-order, 4-stage scheme called SSPRK3/4.
Since there is one additional Runge-Kutta stage, this scheme is more expensive than the variant we've already seen.
*But* the 4-stage scheme has a much larger region of absolute stability, meaning that we can take timesteps that are twice as long.
In the code below, we're dividing the CFL number computed purely from the wave speed by 4 instead of 8 as we did with the previous integration scheme.
Although each step is about 1/3 more expensive, we only need to take half as many steps in total.

In [ ]:
timestep = (δx / 4) / C / (2 * degree + 1)
num_steps = int(final_time / timestep)
print(f'final time: {final_time:5.3f}')
print(f'num steps:  {num_steps}')
dt = final_time / num_steps

output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)
print(f'output frequency: {output_freq}')

In [ ]:
integrator = numerics.SSPRK34(equation, z_0)

In [ ]:
hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

In [ ]:
results['SSPRK3/4, BDFM(2)'] = (hs, qs)

To wrap things up, we'll compare the energy drift of every scheme that we've looked at.

In [ ]:
fig, axes = plt.subplots()
for name, fields in results.items():
    times = np.linspace(0, final_time, len(fields[0]))
    energies = np.array([
        0.5 * assemble((inner(q, q) / h + g * (h + b)) * dx)
        for h, q in zip(*fields)
    ])

    axes.plot(times, energies, label=name)
    
axes.set_ylim(1966.5, 1970)
axes.set_xlabel('time (s)')
axes.set_ylabel('energy (m${}^2$ / s${}^2$)')
axes.legend();

With BDFM elements, the energy drift of the 4-stage scheme is virtually the same as that of the 3-stage scheme, despite needing only half as many timesteps overall.

### Rosenbrock schemes

###### or, how I learned to stop worrying and love implicit methods

Each of the timestepping schemes we've described above is subject to some kind of stability requirement on the timestep.
Some schemes, like SSPRK3/4, can alleviate the burden with larger regions of stability.
We could eliminate these stability concerns entirely by switching to an implicit timestepping scheme like the midpoint method

$$\frac{z_{n + 1} - z_n}{\delta t} = F\left(\frac{z_{n + 1} + z_n}{2}\right).$$

In doing so, we've only introduced another problem: how do we select a solver for the nonlinear system?

*Rosenbrock* schemes make for a middle-ground between explicit and implicit methods.
The key idea of Rosenbrock schemes is to take an implicit scheme, like the midpoint rule written above, and do only one step of Newton's method rather than iterate until ocnvergence.
This involves only a single linear system solve, which is much more work than an explicit method but much less than a full nonlinear solve.
Provided that the timestep isn't so large that there are huge changes to the system state, the old value of the state is probably within the convergence basin for Newton's method and a single step is good enough.

The big advantage of Rosenbrock schemes, as you can see below, is that we can take much, *much* longer timesteps than would be otherwise possible.
Here we're using a timestep that 8x longer than what we used for SSPRK3/4 and 16x longer than the other schemes we showed before.

In [ ]:
timestep = 2 * δx / C / (2 * degree + 1)
num_steps = int(final_time / timestep)
print(f'final time: {final_time:5.3f}')
print(f'num steps:  {num_steps}')
dt = final_time / num_steps

output_time = 1 / 7.5
output_freq = max(int(output_time / dt), 1)
print(f'output frequency: {output_freq}')

In [ ]:
integrator = numerics.RosenbrockMidpoint(equation, z_0)

One thing to observe is that, while we were able to use far fewer timesteps, each step requires much more CPU cycles because the linear system we have to solve is no longer block-diagonal -- it's globally coupled and overall much more complex.
You can see by the timing below that this run took less time than SSPRK3/4.

In [ ]:
hs = []
qs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h, q = z.split()
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
        qs.append(q.copy(deepcopy=True))
    
    integrator.step(dt)

In [ ]:
results['Rosenbrock, BDFM(2)'] = (hs, qs)

The Rosenbrock scheme cuts the energy drift even further.

In [ ]:
fig, axes = plt.subplots()
for name in ['Rosenbrock, BDFM(2)', 'SSPRK3/4, BDFM(2)']:
    fields = results[name]
    times = np.linspace(0, final_time, len(fields[0]))
    energies = np.array([
        0.5 * assemble((inner(q, q) / h + g * (h + b)) * dx)
        for h, q in zip(*fields)
    ])

    axes.plot(times, energies, label=name)
    
axes.set_ylim(1966.5, 1970)
axes.set_xlabel('time (s)')
axes.set_ylabel('energy (m${}^2$ / s${}^2$)')
axes.legend();

The Rosenbrock method is stable at much longer timesteps, but taking very long steps will eventually lead to a degredation in accuracy.
Nonetheless, it reduces or eliminates one of the more aggravating factors in simulating physical systems, especially when spun up from a possibly unrealistic initial state.
While the simulation with this scheme ran faster than the closest competitor (SSPRK3/4), there are several performance upsides we haven't explored yet.
The solver will default to solving the linear system using the iterative GMRES method and an ILU preconditioner.
We could improve on this by using a more specialized preconditioner tailored to this particular problem.